In [2]:
%matplotlib 
#%matplotlib inline
#%matplotlib qt
import os
import csv
import numpy as np
import fnmatch
from pathlib import Path
import pandas as pd
import xlrd
import matplotlib.pyplot as plt
from scipy import stats 
import statsmodels
from statsmodels.graphics.gofplots import qqplot
import pickle
from itertools import *
from operator import *

import itertools


Using matplotlib backend: Qt5Agg


In [3]:
# first find the number of rows in the existing file

#subjectName = 'Per'
subjectName = 'Torben'
#subjectName = 'John'


if subjectName == 'Torben':
    hot_indices = ['1', '0', '2', '1', '2']
else:
    hot_indices = ['1', '2', '1', '2']

# raw data
#filePath_PythonData = r"C:\DTU\Data\PainExperiments\201907_PainTrial2\ExportedData\raw" + "\\" + subjectName + "_wEvents_raw.xlsx"

# iVT filter with fixations
filePath_PythonData = r"C:\DTU\Data\PainExperiments\201907_PainTrial2\ExportedData\IVTfilterAttention" + "\\" + subjectName + "_wEvents_IVTfilterAttention.xlsx"


df_data = pd.read_excel(filePath_PythonData)

In [4]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [10]:
# extract pupil data

pupilRawLeft = df_data['Pupil diameter left [mm]'].tolist()
pupilRawRight = df_data['Pupil diameter right [mm]'].tolist()



pupilRawLeft_woNan = [pupilSize for ind, pupilSize in enumerate(pupilRawLeft) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight[ind])]
pupilRawRight_woNan = [pupilSize for ind, pupilSize in enumerate(pupilRawRight) if not np.isnan(pupilSize) and not np.isnan(pupilRawLeft[ind])]

# extract time 
timeMicroS = df_data['Recording timestamp [μs]'].tolist()
timeMilliS = [time/1000 for time in timeMicroS]
timeS = [time/1000 for time in timeMilliS]

timeMilliS_woNan = [timeMilliS[ind] for ind, pupilSize in enumerate(pupilRawLeft) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight[ind])]


# extract event information
events = df_data['Event'].tolist()

# correct the event information based on the nan values removed

events_wInd = [(ind, event) for ind, event in enumerate(events) if not pd.isnull(event)]

#events_wInd = list()
#for ind, event in events_wInd0:
#    timeEvent, indexEvent = nearestTimePoint(timeMilliS_woNan, timeMilliS[ind])
#    events_wInd.append(tuple((indexEvent, event)))
    

# extract the logged events
indices_loggedEvents_new = [i for i, x in events_wInd if "Logged live Event" in x]

# extract the eye movement types
eyeMovementType = df_data['Eye movement type'].tolist()

eyeMovement_Fixation = [True if i == 'Fixation' else False for i in eyeMovementType]
eyeMovement_Saccade = [True if i == 'Saccade' else False for i in eyeMovementType]
eyeMovement_EyesNotFound = [True if i == 'EyesNotFound' else False for i in eyeMovementType]
eyeMovement_Unclassified = [True if i == 'Unclassified' else False for i in eyeMovementType]


In [7]:
print(set(eyeMovementType))

{nan, 'Fixation', 'Saccade', 'EyesNotFound', 'Unclassified'}


In [11]:
def filterBlinks_pain(pupilDataL, pupilDataR, timeInDatetime):
    # filter any blinks and nan values lasting around 250ms (on average)
    # first the single nan occurances are replaced with mean of the values on either sides, 
    # as they are assumed to be from hardware problems
    # for the rest of the blinks, 250ms before and after the nan values are interpolated with a linear function
    # returns a dataframe with pupil size, and timestamp
    # http://faculty.washington.edu/chudler/facts.html
   
    # create a dataframe from the pupilsize and time
    pupilData_df = pd.DataFrame(list(zip(timeInDatetime, pupilDataL, pupilDataR)), columns=['timeStamp', 'pupilLeft', 'pupilRight'])
    
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (22 samples) before and after the blink will also be removed
    #extraBlinkSamples = 22   
    extraBlinkSamples = 0
    
    # in case of single missing data, that are due to hardware error, replace with the mean of the pupil size before and
    # after the nan value
    # missing values will be the same for left and right pupil
    missingVal_Single_left = np.argwhere(np.isnan(pupilDataL)).tolist()
    missingVal_Single_right = np.argwhere(np.isnan(pupilDataR)).tolist()
    
    missingVal_Single = np.unique(np.array(sorted([index for list_ in [missingVal_Single_left, missingVal_Single_right] for index in list_])))
    
    #missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list 
    
    missingVal_Single_remove = [val for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataL)-1)) if (not np.isnan(pupilDataL[val-1]) and not np.isnan(pupilDataL[val+1])) or ((not np.isnan(pupilDataR[val-1]) and not np.isnan(pupilDataR[val+1])))]
    
    
    # if no blinks present, return the data
    if len(missingVal_Single) == 0:
        print('no nan values present')
        interpolatedNan = np.array([False]*len(pupilData_df['pupilLeft']))
        return pupilData_df, interpolatedNan
    
    # find the index and values to replace for single nan values
    #pupilData_tuples_replaceSingleNan_left = [(val, np.mean([pupilDataL[val-1], pupilDataL[val+1]])) for i, val in enumerate(missingVal_Single_remove) if (val != 0 and val != (len(pupilDataL)-1)) if not np.isnan(pupilDataL[val-1]) and not np.isnan(pupilDataL[val+1])]
    #pupilData_tuples_replaceSingleNan_right = [(val, np.mean([pupilDataR[val-1], pupilDataR[val+1]])) for i, val in enumerate(missingVal_Single_remove) if (val != 0 and val != (len(pupilDataR)-1)) if not np.isnan(pupilDataR[val-1]) and not np.isnan(pupilDataR[val+1])]
    
    pupilData_tuples_replaceSingleNan_left = [(val, np.mean([pupilDataL[val-1], pupilDataL[val+1]])) for i, val in enumerate(missingVal_Single_remove)]
    pupilData_tuples_replaceSingleNan_right = [(val, np.mean([pupilDataR[val-1], pupilDataR[val+1]])) for i, val in enumerate(missingVal_Single_remove)]
    
    
    interpolatedNan = np.array([True if ind in dict(pupilData_tuples_replaceSingleNan_left) else False for ind, val in enumerate(pupilDataL)])
    
    
    # replace the single nan values with the mean of the pupil size on either sides
    indList = -1
    for ind, val in pupilData_tuples_replaceSingleNan_left:
        #print(ind, len(pupilData_df['pupilRight']))
        indList = indList + 1
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilLeft')] = val
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilRight')] = pupilData_tuples_replaceSingleNan_right[indList][1]
        
    
    # again, find the nan values in the pupil size
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index
    # to the next nan value
    
    
    
    # find the nan values again from pupilData['pupilLeft']
    missingVal_Rest_trueFalse_left = pupilData_df['pupilLeft'].isnull()
    missingVal_Rest_trueFalse_right = pupilData_df['pupilRight'].isnull()
    missingVal_Rest_left = [i for i, x in enumerate(missingVal_Rest_trueFalse_left) if x]
    missingVal_Rest_right = [i for i, x in enumerate(missingVal_Rest_trueFalse_right) if x]
    
    
    missingVal_Rest = np.unique(np.array(sorted([index for list_ in [missingVal_Rest_left, missingVal_Rest_right] for index in list_])))
 
    #missingVal_Rest_trueFalse = sorted(missingVal_Rest_trueFalse_left + list(set(missingVal_Rest_trueFalse_right)) - set(missingVal_Rest_trueFalse_left))
    
    
    # if no blinks left, return the current pupilData
    if len(missingVal_Rest) == 0:
        print('no blinks after single nan values removed')
        return pupilData_df, interpolatedNan
    
        
    
    
    
    # in the blinks left, find when the blinks start by finding a difference in the consecutive values of the indices
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    
    blinkStart_tupleList = [(ind, sum(missingVal_RestDifference[0:ind+1])) for ind, val in enumerate(missingVal_RestDifference) if val != 1]
    
    blinkStart_tupleList_wLength = list()
    
    # create a list of tuples of blink start index and the length of the blink
    ind = -1
    blinkLengthSum = 0
    for blink_ind, blinkStartInd in blinkStart_tupleList:
        ind = ind + 1
        if ind != len(blinkStart_tupleList) - 1:
            
            blinkLength = blinkStart_tupleList[ind+1][0]-blink_ind
            blinkLengthSum = blinkLengthSum + blinkLength
            
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
        else:
            # for the last blink -- all blink lengths summed and subtracted from the length of the list
            # missingVal_RestDifference 
            blinkLength = len(missingVal_RestDifference)-blinkLengthSum
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
     
    # add to vector with start and end of tuple
    #beforeAfterNan = [False]*len(pupilData_df['pupilLeft'])
    #for blinkStart, blinkLength in blinkStart_tupleList_wLength:
    #    beforeAfterNan[blinkStart] = True
    #    beforeAfterNan[blinkStart+blinkLength] = True
    #    #print('start and end points: ', pupilData_df['timeStamp'][blinkStart], pupilData_df['timeStamp'][blinkStart + blinkLength])
    
    
    # create lists with start and end values for the blinks, based on blinkStart_tupleList_wLength, regardless of the blink length
    blink_missingData_startList = [blinkStartInd - extraBlinkSamples if (blinkStartInd - extraBlinkSamples) > 0 else 0 for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    blink_missingData_endList = [blinkStartInd + blinkLength + extraBlinkSamples if (blinkStartInd + blinkLength + extraBlinkSamples) < (len(pupilData_df['pupilLeft'])-1) else (len(pupilData_df['pupilLeft'])-1) for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    # create a list of tuples from the start and end points of the blink
    blink_missingData_startEndTuple = [(blinkStart, blink_missingData_endList[ind]) for ind, blinkStart in enumerate(blink_missingData_startList)] 
    
    
    # check if blinks need to be combined - blinksCombine is a list of list of 2 elements, the index of the blinks that should be combined
    blinksCombine = [[ind, ind+1] for ind, blink in enumerate(blink_missingData_startEndTuple[0:-1]) if blink[1] > blink_missingData_startEndTuple[ind+1][0]]
        
    if blinksCombine:
        print('blinks being combined')
        # combine blinks that need to be combined - if multiple consecutive blinks need to be removed: eg - [1, 2], [2, 3] 
        # are included in the blinksCombine, the combined version should be [1, 3] 
        blinksCombineFinal = list()
        ind = -1
        while ind < len(blinksCombine)-2:
            
            ind = ind + 1
            blinkCombining = blinksCombine[ind]
            blinksCombineFinal.append(blinkCombining)
            while ind < len(blinksCombine)-2 and blinkCombining[1] == blinksCombine[ind+1][0]:
                # change the ending of the last added blink of blinksCombineFinal
                blinksCombineFinal[-1][1] = blinksCombine[ind+1][1]
                ind = ind + 1
            
            
        if len(blinksCombine) == 1:
            blinksCombineFinal = blinksCombine.copy()
            
        
        if blinksCombine[-1][1] != blinksCombineFinal[-1][1]:
            if blinksCombine[-1][0] == blinksCombineFinal[-1][1]:
                blinksCombineFinal[-1][1] = blinksCombine[-1][1]
            else:
                blinksCombineFinal.append(blinksCombine[-1])
        
        
        
        
    #    for w, z in groupby(sorted(list(blinksCombine)), lambda x, y=itertools.count(): next(y)-x):
    #        group = list(z)
    #        blinksCombineFinal.append(tuple((group[0], group[-1])))
        
        for x in sorted(blinksCombineFinal, reverse=True):
            new_start = blink_missingData_startEndTuple[x[0]][0] 
            new_end = blink_missingData_startEndTuple[x[1]][1] 
            
            x_start = x[0]
            x_end = x[1]
            
            # delete also the blinkStart_tupleList_wLength, since it is going to be used to compute other metrics
            for blinkRemove in range(x[1], x[0]-1, -1):
                del blink_missingData_startEndTuple[blinkRemove]
            
            blink_missingData_startEndTuple.insert(x[0], tuple((new_start, new_end)))
    
    
    blinkAndNonBlinkDurationList = [length/90 for start, length in blinkStart_tupleList_wLength]
    
    
    # remove blinks from data
    for blinkStart, blinkEnd in blink_missingData_startEndTuple:
        pupilData_df.loc[blinkStart:blinkEnd,'pupilLeft'] = np.nan
        pupilData_df.loc[blinkStart:blinkEnd,'pupilRight'] = np.nan
        replaceTrueList = range(blinkStart, blinkEnd+1, 1)
        interpolatedNan[replaceTrueList] = True
         
    pupilData_df['pupilLeft'] = pupilData_df['pupilLeft'].astype(float).interpolate('linear', limit_direction = 'both')
    pupilData_df['pupilRight'] = pupilData_df['pupilRight'].astype(float).interpolate('linear', limit_direction = 'both')
    
    if pupilData_df.isnull().any().any():
        print('nan values in filtered data')
        #for i,val in enumerate(pupilData_filter[0:5000]):
        #    print(i, val, pupilData_woSingleMissingData[i])
        
    
    return pupilData_df, interpolatedNan

In [12]:
def hampel(dvec, radius=5, nsigma=3, rem_nomed=False):

    # replace outliers with median values (hampel filter)
    mvec = pd.Series(dvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    svec = 1.4862 * np.abs(dvec-mvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    plonk = np.abs(dvec-mvec) > nsigma*svec
    dvec = np.array(dvec)
    dvec[plonk.tolist()] = mvec[plonk.tolist()]

    # remove "bad data" where we cannot calculate a median value due to already missing values
    if (rem_nomed):
        dvec[np.isnan(mvec)] = np.nan
    return dvec


## Check filtering of pupil size using hampel filter

In [13]:
# remove nan values that occur in the other eye
#pupilLeft_equal = [val for ind, val in enumerate(pupilRawLeft_woNanPairs) if ]

# filter the pupil size for pins in the dark using hampel filter 



pupilData_woBlinks, interpolatedValues = filterBlinks_pain(pupilRawLeft, pupilRawRight, timeMilliS)

#pupilData_df_hampel = dict()
    
pupilData_filtered = pupilData_woBlinks.copy()
pupilData_filtered['pupilLeft_hampel'] = hampel(pupilData_woBlinks['pupilLeft'], 25, 3, False)
pupilData_filtered['pupilRight_hampel'] = hampel(pupilData_woBlinks['pupilRight'], 25, 3, False)

pupilData_filtered['timeStamp_plot'] = [time/1000 for time in pupilData_filtered['timeStamp']]

In [14]:
fig = plt.figure()

ax = fig.add_subplot(111)

#ax.plot(timeMilliS, pupilRawLeft, label = 'with Nan')
ax.plot(pupilData_filtered['timeStamp_plot'], pupilRawLeft, label = 'unfiltered left')
ax.plot(pupilData_filtered['timeStamp_plot'], pupilData_filtered['pupilLeft'], label = 'filtered left')

ax.plot(pupilData_woBlinks['timeStamp'], pupilRawRight, '-.', label = 'unfiltered right')
ax.plot(pupilData_woBlinks['timeStamp'], pupilData_woBlinks['pupilRight'], '-.', label = 'filtered right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Absolute pupil size [in mm]')

ax.set_title('Check blink removal filter for the complete experiment')

#pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName + '\\checkFiltering_leftPupil_blinkRemoval', 'wb'))
#pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName + '\\checkFiltering_bothPupil_blinkRemoval', 'wb'))



Text(0.5,1,'Check blink removal filter for the complete experiment')

In [15]:
fig = plt.figure()

ax = fig.add_subplot(111)

#ax.plot(timeMilliS, pupilRawLeft, label = 'with Nan')
ax.plot(pupilData_filtered['timeStamp_plot'], pupilData_filtered['pupilLeft'], label = 'unfiltered')
ax.plot(pupilData_filtered['timeStamp_plot'], pupilData_filtered['pupilLeft_hampel'], '-.', label = 'filtered')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Absolute pupil size [in mm]')

ax.set_title('Check hampel filter for the complete experiment')

#pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName + '\\checkFiltering_leftPupil_hampelFilter', 'wb'))



Text(0.5,1,'Check hampel filter for the complete experiment')

## Pupil sizes from pin in the dark

In [16]:
# indexes of pin events
index_pin_start =  [ind for ind, row in events_wInd if row == 'pin_start']
index_pin_end = [ind for ind, row in events_wInd if row == 'pin_end']


# pupil sizes for pain from pin 
pupilRawLeft_pin_dark = pupilData_filtered['pupilLeft_hampel'][index_pin_start[0]:index_pin_end[0]]
pupilRawRight_pin_dark = pupilData_filtered['pupilRight_hampel'][index_pin_start[0]:index_pin_end[0]]
time_pin_dark = pupilData_filtered['timeStamp'][index_pin_start[0]:index_pin_end[0]]

time_pin_dark_plot = [time/1000 for time in time_pin_dark]

print('Ratio of nan values in this expt from pin is: ', sum(interpolatedValues[index_pin_start[0]:index_pin_end[0]])/len(interpolatedValues[index_pin_start[0]:index_pin_end[0]]))

Ratio of nan values in this expt from pin is:  0.9882001705999431


In [17]:
# find the logged events in the pin events

for index in indices_loggedEvents_new:
    if index > index_pin_start[0]:
        logged_events_start = index
        break
        
for index in sorted(indices_loggedEvents_new, reverse=True):
    if index < index_pin_end[0]:
        logged_events_end = index
        break    
        
indices_loggedEvents_pin_dark = indices_loggedEvents_new[indices_loggedEvents_new.index(logged_events_start):indices_loggedEvents_new.index(logged_events_end)+1]

In [26]:
# eye movements in the expt 

eyeMovement_Fixation_pin_dark = eyeMovement_Fixation[index_pin_start[0]:index_pin_end[0]]
eyeMovement_Saccade_pin_dark = eyeMovement_Saccade[index_pin_start[0]:index_pin_end[0]]
eyeMovement_EyesNotFound_pin_dark = eyeMovement_EyesNotFound[index_pin_start[0]:index_pin_end[0]]
eyeMovement_Unclassified_pin_dark = eyeMovement_Unclassified[index_pin_start[0]:index_pin_end[0]]




In [28]:
# plot the filtered signal for pins in the dark and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_pin_dark_plot, pupilRawLeft_pin_dark, label = 'left')
ax.plot(time_pin_dark_plot, pupilRawRight_pin_dark, label = 'right')

ax.plot(np.array(time_pin_dark_plot)[eyeMovement_Saccade_pin_dark], np.array(pupilRawLeft_pin_dark)[eyeMovement_Saccade_pin_dark], 'x', label = 'Saccades')
ax.plot(np.array(time_pin_dark_plot)[eyeMovement_Unclassified_pin_dark], np.array(pupilRawLeft_pin_dark)[eyeMovement_Unclassified_pin_dark], 'o', label = 'Unclassified')


plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from pins in the dark' %subjectName)

for ind in indices_loggedEvents_pin_dark:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilRawLeft_pin_dark), np.max(pupilRawLeft_pin_dark)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_pinTrial2' + '\\' + subjectName +  '\\pupils_pin_dark_wLogEventsMarked_wEyeMovements', 'wb'))


## Average of individual trials

In [57]:
# average of pin trials

pinTrials_dark_start = indices_loggedEvents_pin_dark[0:-1]
pinTrials_dark_end = indices_loggedEvents_pin_dark[1:]

pinTrial_dark_1 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_dark_start[0]:pinTrials_dark_end[0]].tolist()
pinTrial_dark_2 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_dark_start[1]:pinTrials_dark_end[1]].tolist()
pinTrial_dark_3 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_dark_start[2]:pinTrials_dark_end[2]].tolist()
pinTrial_dark_4 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_dark_start[3]:pinTrials_dark_end[3]].tolist()

timeTrial_dark_1 = np.arange(0, len(pinTrial_dark_1)/100, 1/100)
timeTrial_dark_2 = np.arange(0, len(pinTrial_dark_2)/100, 1/100)
timeTrial_dark_3 = np.arange(0, len(pinTrial_dark_3)/100, 1/100)
timeTrial_dark_4 = np.arange(0, len(pinTrial_dark_4)/100, 1/100)

trialLength_avg_min = np.min([len(pinTrial_dark_1), len(pinTrial_dark_2), len(pinTrial_dark_3), len(pinTrial_dark_4)])

trialLength_avg_minInd = np.argmin([len(pinTrial_dark_1), len(pinTrial_dark_2), len(pinTrial_dark_3), len(pinTrial_dark_4)])

if trialLength_avg_minInd == 0:
    timeTrial_dark_avgPlot = timeTrial_dark_1
elif trialLength_avg_minInd == 1:
    timeTrial_dark_avgPlot = timeTrial_dark_2
elif trialLength_avg_minInd == 2:
    timeTrial_dark_avgPlot = timeTrial_dark_3
else:
    timeTrial_dark_avgPlot = timeTrial_dark_4

pinTrial_dark_avg = np.average([pinTrial_dark_1[0:trialLength_avg_min], pinTrial_dark_2[0:trialLength_avg_min], pinTrial_dark_3[0:trialLength_avg_min], pinTrial_dark_4[0:trialLength_avg_min]], axis = 0)
pinTrial_dark_std = np.std([pinTrial_dark_1[0:trialLength_avg_min], pinTrial_dark_2[0:trialLength_avg_min], pinTrial_dark_3[0:trialLength_avg_min], pinTrial_dark_4[0:trialLength_avg_min]], axis = 0)
pinTrial_dark_stdError = (np.std([pinTrial_dark_1[0:trialLength_avg_min], pinTrial_dark_2[0:trialLength_avg_min], pinTrial_dark_3[0:trialLength_avg_min], pinTrial_dark_4[0:trialLength_avg_min]], axis = 0))/np.sqrt(4)



In [58]:


fig = plt.figure()

ax = fig.add_subplot(111)
ax.plot(timeTrial_dark_1, pinTrial_dark_1, label = 'trial2')
ax.plot(timeTrial_dark_2, pinTrial_dark_2, label = 'trial3')
ax.plot(timeTrial_dark_3[0:len(pinTrial_dark_3)], pinTrial_dark_3, label = 'trial4')
ax.plot(timeTrial_dark_4, pinTrial_dark_4, label = 'trial5')
ax.plot(timeTrial_dark_avgPlot, pinTrial_dark_avg, '--', label = 'mean' )
ax.plot(timeTrial_dark_avgPlot, pinTrial_dark_avg+pinTrial_dark_stdError, ':', label = 'error')
ax.plot(timeTrial_dark_avgPlot, pinTrial_dark_avg-pinTrial_dark_stdError, ':', label = 'error')

ax.fill_between(timeTrial_dark_avgPlot, pinTrial_dark_avg, pinTrial_dark_avg+pinTrial_dark_stdError, color = 'yellow')
ax.fill_between(timeTrial_dark_avgPlot, pinTrial_dark_avg, pinTrial_dark_avg-pinTrial_dark_stdError, color = 'khaki')


plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from pins in the dark' %subjectName)

#pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_pin_dark_trialsAvg_wError', 'wb'))



## Pupil sizes from hot cup in the dark

In [29]:
# indexes for the pain using hot cup in the dark

# indexes of hot events
index_hot_start =  [ind for ind, row in events_wInd if row == 'hot_start']
index_hot_end = [ind for ind, row in events_wInd if row == 'hot_end']


# hand1
# indexes of hot events for hand1
index_hot_start_H1 = [index for ind, index in enumerate(index_hot_start) if hot_indices[ind] == '1'][0]
index_hot_end_H1 = [index for ind, index in enumerate(index_hot_end) if hot_indices[ind] == '1'][0]


# pupil sizes for pain from hot for hand1
pupilRawLeft_hot_dark_H1 = pupilData_filtered['pupilLeft_hampel'][index_hot_start_H1:index_hot_end_H1]
pupilRawRight_hot_dark_H1 = pupilData_filtered['pupilRight_hampel'][index_hot_start_H1:index_hot_end_H1]
time_hot_dark_H1 = pupilData_filtered['timeStamp'][index_hot_start_H1:index_hot_end_H1]

time_hot_dark_plot_H1 = [time/1000 for time in time_hot_dark_H1]

# hand2
# indexes of hot events for hand2
index_hot_start_H2 = [index for ind, index in enumerate(index_hot_start) if hot_indices[ind] == '2'][0]
index_hot_end_H2 = [index for ind, index in enumerate(index_hot_end) if hot_indices[ind] == '2'][0]


# pupil sizes for pain from hot for hand2
pupilRawLeft_hot_dark_H2 = pupilData_filtered['pupilLeft_hampel'][index_hot_start_H2:index_hot_end_H2]
pupilRawRight_hot_dark_H2 = pupilData_filtered['pupilRight_hampel'][index_hot_start_H2:index_hot_end_H2]
time_hot_dark_H2 = pupilData_filtered['timeStamp'][index_hot_start_H2:index_hot_end_H2]

time_hot_dark_plot_H2 = [time/1000 for time in time_hot_dark_H2]

print('ratio of nan values during the hot cup in the dark with the first hand is: ', sum(interpolatedValues[index_hot_start_H1:index_hot_end_H1])/len(interpolatedValues[index_hot_start_H1:index_hot_end_H1]))

print('ratio of nan values during the hot cup in the dark with the second hand is: ', sum(interpolatedValues[index_hot_start_H2:index_hot_end_H2])/len(interpolatedValues[index_hot_start_H2:index_hot_end_H2]))


ratio of nan values during the hot cup in the dark with the first hand is:  0.9930875576036866
ratio of nan values during the hot cup in the dark with the second hand is:  0.9944584382871536


In [30]:
# find the logged events in the hot events in the dark

# hand1
for index in indices_loggedEvents_new:
    if index > index_hot_start_H1:
        logged_events_start_H1 = index
        break
        
for index in sorted(indices_loggedEvents_new, reverse=True):
    if index < index_hot_end_H1:
        logged_events_end_H1 = index
        break

indices_loggedEvents_hot_dark_H1 = indices_loggedEvents_new[indices_loggedEvents_new.index(logged_events_start_H1):indices_loggedEvents_new.index(logged_events_end_H1)+1]


# hand2
for index in indices_loggedEvents_new:
    if index > index_hot_start_H2:
        logged_events_start_H2 = index
        break
        
for index in sorted(indices_loggedEvents_new, reverse=True):
    if index < index_hot_end_H2:
        logged_events_end_H2 = index
        break
        
indices_loggedEvents_hot_dark_H2 = indices_loggedEvents_new[indices_loggedEvents_new.index(logged_events_start_H2):indices_loggedEvents_new.index(logged_events_end_H2)+1]


In [31]:
# eye movements in the expt 

eyeMovement_Fixation_hot_dark = eyeMovement_Fixation[index_hot_start_H2:index_hot_end_H2]
eyeMovement_Saccade_hot_dark = eyeMovement_Saccade[index_pin_start[0]:index_pin_end[0]]
eyeMovement_EyesNotFound_hot_dark = eyeMovement_EyesNotFound[index_pin_start[0]:index_pin_end[0]]
eyeMovement_Unclassified_hot_dark = eyeMovement_Unclassified[index_pin_start[0]:index_pin_end[0]]




In [33]:
# plot the filtered signal for hot cup for hand1 in the dark and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_hot_dark_plot_H1, pupilRawLeft_hot_dark_H1, label = 'left')
ax.plot(time_hot_dark_plot_H1, pupilRawRight_hot_dark_H1, label = 'right')

ax.plot(np.array(time_hot_dark_plot_H1)[eyeMovement_Saccade_hot_dark], np.array(pupilRawLeft_hot_dark_H1)[eyeMovement_Saccade_hot_dark], 'x', label = 'Saccades')
ax.plot(np.array(time_hot_dark_plot_H1)[eyeMovement_Unclassified_hot_dark], np.array(pupilRawLeft_hot_dark_H1)[eyeMovement_Unclassified_hot_dark], 'o', label = 'Unclassified')


plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from hot cup for hand1 in the dark' %subjectName)

for ind in indices_loggedEvents_hot_dark_H1:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilRawLeft_hot_dark_H1), np.max(pupilRawLeft_hot_dark_H1)], '-.', color = 'gray')

#pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_hot_dark_wLogEventsMarked_H1', 'wb'))


IndexError: boolean index did not match indexed array along dimension 0; dimension is 2170 but corresponding boolean dimension is 7034

In [318]:
# plot the filtered signal for hot cup for hand2 in the dark and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_hot_dark_plot_H2, pupilRawLeft_hot_dark_H2, label = 'left')
ax.plot(time_hot_dark_plot_H2, pupilRawRight_hot_dark_H2, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from hot cup for hand2 in the dark' %subjectName)

for ind in indices_loggedEvents_hot_dark_H2:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilRawLeft_hot_dark_H2), np.max(pupilRawLeft_hot_dark_H2)], '-.', color = 'gray')

#pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_hot_dark_wLogEventsMarked_H2', 'wb'))


## Pupil sizes from pins in light

In [77]:

# pupil sizes for pain from pin 
pupilRawLeft_pin_light = pupilData_filtered['pupilLeft_hampel'][index_pin_start[-1]:index_pin_end[-1]]
pupilRawRight_pin_light = pupilData_filtered['pupilRight_hampel'][index_pin_start[-1]:index_pin_end[-1]]
time_pin_light = pupilData_filtered['timeStamp'][index_pin_start[-1]:index_pin_end[-1]]


time_pin_light_plot = [time/1000 for time in time_pin_light]

print('Ratio of nan values in this expt from pin is: ', sum(interpolatedValues[index_pin_start[-1]:index_pin_end[-1]])/len(interpolatedValues[index_pin_start[-1]:index_pin_end[-1]]))

Ratio of nan values in this expt from pin is:  0.9879288437102922


In [46]:
for i,x in enumerate(interpolatedValues[index_pin_start[-1]:index_pin_end[-1]]):
    print(i, x)

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True
20 True
21 True
22 True
23 True
24 True
25 True
26 True
27 True
28 True
29 True
30 True
31 True
32 True
33 True
34 True
35 True
36 True
37 True
38 False
39 True
40 True
41 True
42 True
43 True
44 True
45 True
46 True
47 True
48 True
49 True
50 True
51 True
52 True
53 True
54 True
55 True
56 True
57 True
58 True
59 True
60 True
61 True
62 True
63 True
64 True
65 True
66 True
67 True
68 True
69 True
70 True
71 True
72 True
73 True
74 True
75 True
76 True
77 True
78 True
79 True
80 True
81 True
82 True
83 True
84 True
85 True
86 True
87 True
88 True
89 True
90 True
91 True
92 True
93 True
94 True
95 True
96 True
97 True
98 True
99 True
100 True
101 True
102 True
103 True
104 True
105 True
106 True
107 True
108 True
109 True
110 True
111 True
112 True
113 True
114 True
115 True
116 True
117 True
118 True
119 False
120 True
121 True
122 True
1

1583 False
1584 True
1585 True
1586 True
1587 True
1588 True
1589 True
1590 True
1591 True
1592 True
1593 True
1594 True
1595 True
1596 True
1597 True
1598 True
1599 True
1600 True
1601 True
1602 True
1603 True
1604 True
1605 True
1606 True
1607 True
1608 True
1609 True
1610 True
1611 True
1612 True
1613 True
1614 True
1615 True
1616 True
1617 True
1618 True
1619 True
1620 True
1621 True
1622 True
1623 True
1624 True
1625 True
1626 True
1627 True
1628 True
1629 True
1630 True
1631 True
1632 True
1633 True
1634 True
1635 True
1636 True
1637 True
1638 True
1639 True
1640 True
1641 True
1642 True
1643 True
1644 True
1645 True
1646 True
1647 True
1648 True
1649 True
1650 True
1651 True
1652 True
1653 True
1654 True
1655 True
1656 True
1657 True
1658 True
1659 True
1660 True
1661 True
1662 True
1663 False
1664 True
1665 True
1666 True
1667 True
1668 True
1669 True
1670 True
1671 True
1672 True
1673 True
1674 True
1675 True
1676 True
1677 True
1678 True
1679 True
1680 True
1681 True
1682 Tru

2583 True
2584 True
2585 True
2586 True
2587 True
2588 True
2589 True
2590 True
2591 True
2592 True
2593 True
2594 True
2595 True
2596 True
2597 True
2598 True
2599 True
2600 True
2601 True
2602 True
2603 True
2604 True
2605 True
2606 True
2607 True
2608 True
2609 True
2610 True
2611 True
2612 True
2613 True
2614 True
2615 True
2616 True
2617 True
2618 True
2619 True
2620 True
2621 True
2622 True
2623 True
2624 True
2625 True
2626 True
2627 True
2628 True
2629 True
2630 True
2631 True
2632 True
2633 True
2634 True
2635 True
2636 True
2637 True
2638 True
2639 False
2640 True
2641 True
2642 True
2643 True
2644 True
2645 True
2646 True
2647 True
2648 True
2649 True
2650 True
2651 True
2652 True
2653 True
2654 True
2655 True
2656 True
2657 True
2658 True
2659 True
2660 True
2661 True
2662 True
2663 True
2664 True
2665 True
2666 True
2667 True
2668 True
2669 True
2670 True
2671 True
2672 True
2673 True
2674 True
2675 True
2676 True
2677 True
2678 True
2679 True
2680 True
2681 True
2682 True

4332 True
4333 True
4334 True
4335 True
4336 True
4337 True
4338 True
4339 True
4340 True
4341 True
4342 True
4343 True
4344 True
4345 True
4346 True
4347 True
4348 True
4349 True
4350 True
4351 True
4352 True
4353 True
4354 True
4355 True
4356 True
4357 True
4358 True
4359 True
4360 True
4361 True
4362 True
4363 True
4364 True
4365 True
4366 True
4367 True
4368 True
4369 True
4370 True
4371 True
4372 True
4373 False
4374 True
4375 True
4376 True
4377 True
4378 True
4379 True
4380 True
4381 True
4382 True
4383 True
4384 True
4385 True
4386 True
4387 True
4388 True
4389 True
4390 True
4391 True
4392 True
4393 True
4394 True
4395 True
4396 True
4397 True
4398 True
4399 True
4400 True
4401 True
4402 True
4403 True
4404 True
4405 True
4406 True
4407 True
4408 True
4409 True
4410 True
4411 True
4412 True
4413 True
4414 True
4415 True
4416 True
4417 True
4418 True
4419 True
4420 True
4421 True
4422 True
4423 True
4424 True
4425 True
4426 True
4427 True
4428 True
4429 True
4430 True
4431 True

5832 True
5833 True
5834 True
5835 True
5836 True
5837 True
5838 True
5839 True
5840 True
5841 True
5842 True
5843 True
5844 True
5845 True
5846 True
5847 True
5848 True
5849 True
5850 True
5851 True
5852 True
5853 True
5854 True
5855 True
5856 True
5857 True
5858 True
5859 True
5860 True
5861 True
5862 True
5863 True
5864 True
5865 True
5866 True
5867 True
5868 True
5869 True
5870 True
5871 True
5872 True
5873 True
5874 True
5875 True
5876 True
5877 True
5878 True
5879 True
5880 True
5881 True
5882 True
5883 True
5884 True
5885 True
5886 True
5887 True
5888 True
5889 True
5890 True
5891 True
5892 True
5893 True
5894 True
5895 True
5896 True
5897 True
5898 True
5899 True
5900 True
5901 True
5902 True
5903 True
5904 True
5905 True
5906 True
5907 True
5908 True
5909 True
5910 True
5911 True
5912 True
5913 True
5914 False
5915 True
5916 True
5917 True
5918 True
5919 True
5920 True
5921 True
5922 True
5923 True
5924 True
5925 True
5926 True
5927 True
5928 True
5929 True
5930 True
5931 True

7331 True
7332 True
7333 True
7334 True
7335 True
7336 True
7337 True
7338 True
7339 True
7340 True
7341 True
7342 True
7343 True
7344 True
7345 True
7346 True
7347 True
7348 True
7349 True
7350 True
7351 True
7352 True
7353 True
7354 False
7355 True
7356 True
7357 True
7358 True
7359 True
7360 True
7361 True
7362 True
7363 True
7364 True
7365 True
7366 True
7367 True
7368 True
7369 True
7370 True
7371 True
7372 True
7373 True
7374 True
7375 True
7376 True
7377 True
7378 True
7379 True
7380 True
7381 True
7382 True
7383 True
7384 True
7385 True
7386 True
7387 True
7388 True
7389 True
7390 True
7391 True
7392 True
7393 True
7394 True
7395 True
7396 True
7397 True
7398 True
7399 True
7400 True
7401 True
7402 True
7403 True
7404 True
7405 True
7406 True
7407 True
7408 True
7409 True
7410 True
7411 True
7412 True
7413 True
7414 True
7415 True
7416 True
7417 True
7418 True
7419 True
7420 True
7421 True
7422 True
7423 True
7424 True
7425 True
7426 True
7427 True
7428 True
7429 True
7430 True

In [40]:
# find the logged events in the pin events

for index in indices_loggedEvents_new:
    if index > index_pin_start[-1]:
        logged_events_start = index
        break
        
for index in sorted(indices_loggedEvents_new, reverse=True):
    if index < index_pin_end[-1]:
        logged_events_end = index
        break
        
indices_loggedEvents_pin_light = indices_loggedEvents_new[indices_loggedEvents_new.index(logged_events_start):indices_loggedEvents_new.index(logged_events_end)+1]

In [41]:
# plot the filtered signal for pins in the light and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_pin_light_plot, pupilRawLeft_pin_light, label = 'left')
ax.plot(time_pin_light_plot, pupilRawRight_pin_light, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from pins in the light' %subjectName)

for ind in indices_loggedEvents_pin_light:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilRawLeft_pin_light), np.max(pupilRawLeft_pin_light)], '-.', color = 'gray')

#pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_pin_light_wLogEventsMarked', 'wb'))


In [26]:
# average of pin trials

pinTrials_light_start = indices_loggedEvents_pin_light[0:-1]
pinTrials_light_end = indices_loggedEvents_pin_light[1:]

pinTrial_light_1 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_light_start[0]:pinTrials_light_end[0]]
pinTrial_light_2 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_light_start[1]:pinTrials_light_end[1]].tolist()
pinTrial_light_3 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_light_start[2]:pinTrials_light_end[2]].tolist()
pinTrial_light_4 =  pupilData_filtered['pupilLeft_hampel'][pinTrials_light_start[3]:pinTrials_light_end[3]].tolist()

timeTrial_light_1 = np.arange(0, len(pinTrial_light_1)/100, 1/100)
timeTrial_light_2 = np.arange(0, len(pinTrial_light_2)/100, 1/100)
timeTrial_light_3 = np.arange(0, len(pinTrial_light_3)/100, 1/100)
timeTrial_light_4 = np.arange(0, len(pinTrial_light_4)/100, 1/100)


trialLength_avg_min = np.min([len(pinTrial_light_1), len(pinTrial_light_2), len(pinTrial_light_3), len(pinTrial_light_4)])

trialLength_avg_minInd = np.argmin([len(pinTrial_light_1), len(pinTrial_light_2), len(pinTrial_light_3), len(pinTrial_light_4)])

if trialLength_avg_minInd == 0:
    timeTrial_light_avgPlot = timeTrial_light_1
elif trialLength_avg_minInd == 1:
    timeTrial_light_avgPlot = timeTrial_light_2
elif trialLength_avg_minInd == 2:
    timeTrial_light_avgPlot = timeTrial_light_3
else:
    timeTrial_light_avgPlot = timeTrial_light_4
    
pinTrial_light_avg = np.average([pinTrial_light_1[0:trialLength_avg_min], pinTrial_light_2[0:trialLength_avg_min], pinTrial_light_3[0:trialLength_avg_min], pinTrial_light_4[0:trialLength_avg_min]], axis = 0)
pinTrial_light_std = np.std([pinTrial_light_1[0:trialLength_avg_min], pinTrial_light_2[0:trialLength_avg_min], pinTrial_light_3[0:trialLength_avg_min], pinTrial_light_4[0:trialLength_avg_min]], axis = 0)
pinTrial_light_stdError = (np.std([pinTrial_light_1[0:trialLength_avg_min], pinTrial_light_2[0:trialLength_avg_min], pinTrial_light_3[0:trialLength_avg_min], pinTrial_light_4[0:trialLength_avg_min]], axis = 0))/np.sqrt(4)



In [42]:


fig = plt.figure()

ax = fig.add_subplot(111)
ax.plot(timeTrial_light_1, pinTrial_light_1, label = 'trial2')
ax.plot(timeTrial_light_2, pinTrial_light_2, label = 'trial3')
ax.plot(timeTrial_light_3, pinTrial_light_3, label = 'trial4')
ax.plot(timeTrial_light_4, pinTrial_light_4, label = 'trial5')
ax.plot(timeTrial_light_avgPlot, pinTrial_light_avg, '--', label = 'mean' )
ax.plot(timeTrial_light_avgPlot, pinTrial_light_avg+pinTrial_light_stdError, ':', label = 'error')
ax.plot(timeTrial_light_avgPlot, pinTrial_light_avg-pinTrial_light_stdError, ':', label = 'error')

ax.fill_between(timeTrial_light_avgPlot, pinTrial_light_avg, pinTrial_light_avg+pinTrial_light_stdError, color = 'yellow')
ax.fill_between(timeTrial_light_avgPlot, pinTrial_light_avg, pinTrial_light_avg-pinTrial_light_stdError, color = 'khaki')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from pins in the light' %subjectName)

#pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_pin_light_trialsAvg_wError', 'wb'))



## Pupil sizes from hot cup in the light

In [78]:
# indexes for the pain using hot cup in the light

# indexes of hot events
index_hot_start =  [ind for ind, row in events_wInd if row == 'hot_start']
index_hot_end = [ind for ind, row in events_wInd if row == 'hot_end']


# hand1
# indexes of hot events for hand1
index_hot_start_H1 = [index for ind, index in enumerate(index_hot_start) if hot_indices[ind] == '1'][-1]
index_hot_end_H1 = [index for ind, index in enumerate(index_hot_end) if hot_indices[ind] == '1'][-1]

# pupil sizes for pain from hot for hand1
pupilRawLeft_hot_light_H1 = pupilData_filtered['pupilLeft_hampel'][index_hot_start_H1:index_hot_end_H1]
pupilRawRight_hot_light_H1 = pupilData_filtered['pupilRight_hampel'][index_hot_start_H1:index_hot_end_H1]
time_hot_light_H1 = pupilData_filtered['timeStamp'][index_hot_start_H1:index_hot_end_H1]


time_hot_light_plot_H1 = [time/1000 for time in time_hot_light_H1]


# hand2
# indexes of hot events for hand2
index_hot_start_H2 = [index for ind, index in enumerate(index_hot_start) if hot_indices[ind] == '2'][-1]
index_hot_end_H2 = [index for ind, index in enumerate(index_hot_end) if hot_indices[ind] == '2'][-1]

# pupil sizes for pain from hot for hand2
pupilRawLeft_hot_light_H2 = pupilData_filtered['pupilLeft_hampel'][index_hot_start_H2:index_hot_end_H2]
pupilRawRight_hot_light_H2 = pupilData_filtered['pupilRight_hampel'][index_hot_start_H2:index_hot_end_H2]
time_hot_light_H2 = pupilData_filtered['timeStamp'][index_hot_start_H2:index_hot_end_H2]

time_hot_light_plot_H2 = [time/1000 for time in time_hot_light_H2]


print('ratio of nan values during the hot cup in the light with the first hand is: ', sum(interpolatedValues[index_hot_start_H1:index_hot_end_H1])/len(interpolatedValues[index_hot_start_H1:index_hot_end_H1]))

print('ratio of nan values during the hot cup in the light with the second hand is: ', sum(interpolatedValues[index_hot_start_H2:index_hot_end_H2])/len(interpolatedValues[index_hot_start_H2:index_hot_end_H2]))


ratio of nan values during the hot cup in the light with the first hand is:  0.9885920436817472
ratio of nan values during the hot cup in the light with the second hand is:  0.9881132483250487


In [325]:
# find the logged events in the hot events in the light

# hand1
for index in indices_loggedEvents_new:
    if index > index_hot_start_H1:
        logged_events_start_H1 = index
        break
        
for index in sorted(indices_loggedEvents_new, reverse=True):
    if index < index_hot_end_H1:
        logged_events_end_H1 = index
        break

indices_loggedEvents_hot_light_H1 = indices_loggedEvents_new[indices_loggedEvents_new.index(logged_events_start_H1):indices_loggedEvents_new.index(logged_events_end_H1)+1]


# hand2
for index in indices_loggedEvents_new:
    if index > index_hot_start_H2:
        logged_events_start_H2 = index
        break
        
for index in sorted(indices_loggedEvents_new, reverse=True):
    if index < index_hot_end_H2:
        logged_events_end_H2 = index
        break
        
indices_loggedEvents_hot_light_H2 = indices_loggedEvents_new[indices_loggedEvents_new.index(logged_events_start_H2):indices_loggedEvents_new.index(logged_events_end_H2)+1]


In [326]:
# plot the filtered signal for hot cup for hand1 in the light and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_hot_light_plot_H1, pupilRawLeft_hot_light_H1, label = 'left')
ax.plot(time_hot_light_plot_H1, pupilRawRight_hot_light_H1, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from hot cup for hand1 in the light' %subjectName)

for ind in indices_loggedEvents_hot_light_H1:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilRawLeft_hot_light_H1), np.max(pupilRawRight_hot_light_H1)], '-.', color = 'gray')

#pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_hot_light_wLogEventsMarked_H1', 'wb'))


# plot the filtered signal for hot cup for hand2 in the light and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_hot_light_plot_H2, pupilRawLeft_hot_light_H2, label = 'left')
ax.plot(time_hot_light_plot_H2, pupilRawRight_hot_light_H2, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from hot cup for hand2 in the light' %subjectName)

for ind in indices_loggedEvents_hot_light_H2:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilRawLeft_hot_light_H2), np.max(pupilRawLeft_hot_light_H2)], '-.', color = 'gray')

#pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_hot_light_wLogEventsMarked_H2', 'wb'))
